En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# Prefacio

## Metodos

El analisis puede hacerse localmente y directamente, o subiendo el JSON a un cloud service provider y hacerlo a traves de el. Esto presenta muchas ventajas, tales como tener a disposicion todos los recursos de por ejemplo Google Cloud Platform (GCP). Si bien subir el archivo tomara un poco de tiempo y memoria, una vez subido, todos los analisis posteriores seran mucho mas veloces y ligeros de memoria. Por supuesto, se tendra que pagar el servicio utilizado lo cual es algo a tener en cuenta y ademas, el tiempo de enviar y recibir los mensajes del local a la nube causaran un tiempo extra de demora.

Esta estrategia tambien puede hacerse localmente, por ejemplo creando una base de datos local `.sqlite` a partir del JSON, e introduciendo indices para acelerar las busquedas. Nuevamente, este paso incial de crear la db costara algo de tiempo y memoria, pero una vez hecho, los siguientes analisis seran mucho mas rapidos y ligeros. Hacerlo localmente evita la demora de tener que comunicarse con la nube, pero no nos permite tener todos los beneficios de ella.

He puesto un simple proof-of-concept para este metodo .sqlite en el archivo `q1_sqlite`.

Para la optimizacion por tiempo utilizo los metodos directos de python sin la nube, ya que terminan siendo mas rapidos sin la demora por mensajes por internet. Para la optimizacion por memoria, utilizo los metodos de la nube, usando GCP BigQuery.

Para estos utlimos, deberan crear su propio projecto, dataset, y credenciales. Las credenciales se crean en GCP y son .json. Del dataset y projecto se necesitan sus IDs. En el archivo gcp_credentials.py se deben cargar esas IDs, la locacion del archivo de credenciales .json, y el nombre que se dara a la tabla.

Los metodos, tal como pide el challenge, requieren el file path del .json de los tweets el cual debe haber sido descargado.

## Twitter

La estructura de datos del .json [provisto](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) no es exactamente igual a la vista en la pagina de la [documentación oficial de twitter](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object). Por ejemplo, en vez de `date` se utiliza `created_at`. Ante esta discrepancia, opto por utilizar la estructura del .json provisto

## GitFlow

La pagina de [GitFlow](https://www.atlassian.com/git/tutorials/comparing-workflows/gitflow-workflow) dice que es un flow legacy, que ha caido en desuso y solo se muetsra por motivos historicos. Utilizo algunas de sus practicas (una rama main y una develop) pero fuera de eso me apego a simplemente tener ramas para cada feature que luego seran mergeadas a develop y luego a main.

## Documentacion

Sigo la idea de que el codigo debe ser auto-documentante. Aun asi, agrego comentarios con explicaciones breves para seguir las instrucciones del challenge.
Las variables son llamadas en ingle, como es mejor practica, pero los mensajes y comentarios los mantengo en español para apegarme al idioma del challenge.

## Analisis

Utilizo cProfiler para medir el tiempo y memory_profiler para la memoria.

## q1_time

Luego de varias pruebas, encontre que el metodo de lectura mas veloz y ligero es linea-por-linea, aprovechando que el JSON es newline-delimited. La libreria de json mas veloz que halle fue `orjson`. Finalmente, logre un tiempo de aprox. 2 segundos, en el cual el cuello de botella es la lectura JSON. Otras optimizaciones de codigo (como concurrent.futures, datasets de pandas, tratar de cargar todo el JSON en memoria directamente, o en batches, etc) fueron menos exitosas que esta.

A ir leyendo linea por linea, agrego cada fecha (sin hora) como key en un diccionario, una estructura muy eficaz para este problema. Luego a dicha key le agrego el contador ("total") en 1, y le agrego a sub diccionario su username, tambien agregandole el contador en 1. Asi, al final, tendre un dict con cada fecha, su cantidas, y sus usuarios y cantidades, para finalmente ordenarlos y mostrarlos.

In [21]:
from q1_time import q1_time

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q1_time(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[('2021-02-12', 'RanbirS00614606'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-14', 'rebelpacifist'), ('2021-02-18', 'neetuanjle_nitu'), ('2021-02-15', 'jot__b'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-23', 'Surrypuria'), ('2021-02-19', 'Preetm91')]
         822098 function calls (822097 primitive calls) in 2.326 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    1.135    0.000    1.135    0.000 {orjson.loads}
   117408    0.638    0.000    1.773    0.000 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\stream_json.py:5(stream_json)
        1    0.337    0.337    2.317    2.317 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\q1_time.py:7(q1_time)
   234814    0.087    0.000    0.087    0.000 {built-in method sys.intern}
   117407    0.067    0.000    0.067    0.000 {method 'isoformat' of 'datetime.date' o

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     8     52.5 MiB     52.5 MiB           1   @profile
     9                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    10                                             """Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por
    11                                             cada uno de esos días."""
    12     52.5 MiB      0.0 MiB           1       if file_path is None or file_path == "":
    13                                                 print("Error: File path vacio.")
    14                                                 return []
    15                                             # Un limite para que se detenga la lectura del archivo si hay demasiados errores
    16                                             # maxErrorCount se podria poner como variable opcional, o obtenerla de una variable de entorno 
    17     52.5 MiB      0.0 MiB           1       errorCount = 0
    18     52.5 MiB      0.0 MiB           1       maxErrorCount = 100
    19
    20                                             # Utilizaremos un dict para ir creando una key por cada dia, y para el almacenar el contador y sus usernames
    21                                             # Esta estructura permite un rapido acceso a los datos cuando se requiera cargar mas en la misma fecha
    22                                             # Lo mismo para el contador de username, otro dict.
    23     57.7 MiB      0.0 MiB          27       tweets_per_day = defaultdict(lambda: {"total": 0, "users": defaultdict(int)})
    24
    25     58.1 MiB    -48.7 MiB      117408       for tweet in stream_json(file_path):
    26     58.1 MiB    -52.9 MiB      117407           try:
    27                                                     # Utilizamos intern para reducir el uso de memoria (aunque es minima la mejora)        
    28                                                     # Incrementamos el contador de tweets por dia y por usuario
    29     58.1 MiB    -52.9 MiB      117407               date = intern(datetime.fromisoformat(tweet["date"]).date().isoformat())
    30     58.1 MiB    -52.9 MiB      117407               tweets_per_day[date]["total"] += 1
    31
    32     58.1 MiB    -52.2 MiB      117407               username = intern(tweet["user"]["username"])
    33     58.1 MiB    -52.1 MiB      117407               tweets_per_day[date]["users"][username] += 1
    34
    35                                                 except Exception as e:
    36                                                     if isinstance(e, KeyError):
    37                                                         print(f"Error: Key no encontrado en tweet. Saltando linea. {e}")
    38                                                     elif isinstance(e, ValueError):
    39                                                         print(f"Error: Formato invalido en tweet. Saltando linea. {e}")
    40                                                     elif isinstance(e, TypeError):
    41                                                         print(f"Error: Tipo invalido en tweet. Saltando linea. {e}")
    42                                                     else:
    43                                                         print(f"Error: {e}")
    44                                                     errorCount += 1
    45                                                     if (errorCount >= maxErrorCount):
    46                                                         print(f"Demasiados errores, abortando...")
    47                                                         break
    48
    49                                             # Ordenamos las fechas por el total de tweets en order desc y tomamos las 10 primeras
    50     58.1 MiB      0.0 MiB          27       top_dates = sorted(tweets_per_day.keys(), key=lambda x: tweets_per_day[x]["total"], reverse=True)[:10]
    51                                             # Para cada fecha, obtenemos el usuario con mas tweets, y formateamos los datos como se pide   
    52     58.1 MiB      0.0 MiB          11       result = [(date, max(tweets_per_day[date]["users"], key=tweets_per_day[date]["users"].get)) for date in top_dates]
    53     58.1 MiB      0.0 MiB           1       return result

## q1_memory

El JSON, la primera vez que se corre uno de los metodos de la nube, es subido a una tabla en GCP (debe haberse creado el projecto y dataset). Una vez subido, todas las posteriores llamadas utilizaran esa data desde la nube. Este proceso me tardo aprox. 20 segundos.

Hago un preprocesado el JSON, extrayendo solo los campos relevantes para los 3 ejercisios, en formatos convenientes (`DATE` para fechas, `STRING` para el resto, con los mentioned users y emojis siendo Arrays. Esto permite minimzar el tiempo y ancho de banda gastado en subirlos a GCP, y el gasto de almacenamiento alli. Y sus nuevas etsructuras aceleran las queries.

Si se desean hacer luego otros analisis, se puede extender el pre procesado para incluir otros campos (en otra tabla).

In [17]:
from q1_memory import q1_memory

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q1_memory(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
         18356 function calls (18190 primitive calls) in 4.137 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    2.375    1.188    2.375    1.188 {method 'do_handshake' of '_ssl._SSLSocket' objects}
        9    1.617    0.180    1.617    0.180 {method 'read' of '_ssl._SSLSocket' objects}
        2    0.042    0.021    0.042    0.021 {built-in method _socket.getaddrinfo}
        1    0.028    0.028    0.028    0.028 C:\Users\federico.gantov\AppData\Local\Programs\

Los resultados en la consola de GCP (que ignoran el tiempo de mensajeo local-nube) son:

Duration: 0 sec
Bytes processed: 2.45 MB
Bytes billed: 10 MB
Slot milliseconds: 25206

## q2_time

Principalmente igual al q1_time. Pero en este caso el analisis de emojis es el cuello de botella. Aunque puede hacerse mas rapidamente utilizando una expresion regex (moderadamente compleja) en lugar de la libreria emoji, debido a que los emojis estan en cosntante evolucion, es mucho mas robusto utilizar una libreria que se sabe estara actualizada.

In [15]:
from q2_time import q2_time

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q2_time(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
         17622026 function calls (17622018 primitive calls) in 7.089 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    2.987    0.000    5.110    0.000 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\extract_emojis.py:7(extract_emojis)
 17034083    2.024    0.000    2.024    0.000 C:\Users\federico.gantov\AppData\Local\Programs\Python\Python312\Lib\site-packages\emoji\core.py:325(is_emoji)
   117407    1.132    0.000    1.132    0.000 {orjson.loads}
   117408    0.647    0.000    1.779    0.000 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\stream_json.py:5(stream_json)
        1    0.196    0.196    7.078    7.078 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\q2_time.py:6(q2_time)
   117407    0.055    0.000    0.05

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     7     60.9 MiB     60.9 MiB           1   @profile
     8                                         def q2_time(file_path: str) -> List[Tuple[str, int]]:
     9                                             """Los top 10 emojis más usados con su respectivo conteo."""
    10     60.9 MiB      0.0 MiB           1       if file_path is None or file_path == "":
    11                                                 print("Error: File path vacio.")
    12                                                 return []
    13                                             # Un limite para que se detenga la lectura del archivo si hay demasiados errores
    14                                             # maxErrorCount se podria poner como variable opcional, o obtenerla de una variable de entorno 
    15     60.9 MiB      0.0 MiB           1       errorCount = 0
    16     60.9 MiB      0.0 MiB           1       maxErrorCount = 5
    17
    18                                             # Utilizaremos un dict para ir creando una key por cada emoji, y para el almacenar el contador 
    19                                             # Esta estructura permite un rapido acceso a los datos cuando se requiera cargar mas en el mismo emoji
    20     60.9 MiB      0.0 MiB           1       emoji_counts = defaultdict(int)
    21
    22     62.4 MiB   -256.8 MiB      117408       for tweet in stream_json(file_path):
    23     62.4 MiB   -258.3 MiB      117407           try:
    24                                                     # Utilizar intern para content no es util ya que seran muy diferentes
    25     62.4 MiB   -258.3 MiB      117407               content = tweet["content"]
    26                                                     # Incrementamos el contador de emojis
    27     62.4 MiB   -349.4 MiB      163043               for em in extract_emojis(content):
    28     62.4 MiB    -91.1 MiB       45636                   emoji_counts[em] += 1
    29
    30                                                 except Exception as e:
    31                                                     if isinstance(e, KeyError):
    32                                                         print(f"{errorCount} Error: Key no encontrado en tweet. Saltando linea. {e}")      
    33                                                     elif isinstance(e, ValueError):
    34                                                         print(f"{errorCount} Error: Formato invalido en tweet. Saltando linea. {e}")       
    35                                                     elif isinstance(e, TypeError):
    36                                                         print(f"{errorCount} Error: Tipo invalido en tweet. Saltando linea. {e}")
    37                                                     else:
    38                                                         print(f"{errorCount} Error: {e}")
    39                                                     errorCount += 1
    40                                                     if (errorCount >= maxErrorCount):
    41                                                         print(f"{errorCount} Demasiados errores, abortando...")
    42                                                         break
    43
    44                                             # Ordenamos los emojis por cantidad y tomamos los 10 primeros, y retornamos en el formato pedido
    45     62.5 MiB      0.0 MiB        1283       top_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    46     62.5 MiB      0.0 MiB           1       return top_emojis

## q2_memory

La misma idea que q1_memory. Es importante aseguar la correcta decodificacion de los emojis, ya que estan en formato surrogate pairs.

In [18]:
from q2_memory import q2_memory

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q2_memory(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]
         17955 function calls (17794 primitive calls) in 3.998 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    2.052    1.026    2.052    1.026 C:\Users\federico.gantov\AppData\Local\Programs\Python\Python312\Lib\selectors.py:313(_select)
        8    1.481    0.185    1.481    0.185 {method 'read' of '_ssl._SSLSocket' objects}
        2    0.382    0.191    0.382    0.191 {method 'do_handshake' of '_ssl._SSLSocket' objects}
        2    0.029    0.014    0.029    0.014 {built-in method _socket.getaddrinfo}
        2    0.026    0.013    0.026    0.013 {built-in method builtins.pow}
        2    0.002    0.001    0.002    0.001 C:\Users\federico.gantov\AppData\Local\Programs\Python\Python312\Lib\site-packages\rsa\common.py:105(extended_gcd)
        9    0.001    0.000    0.001    0.

Los resultados en la consola de GCP (que ignoran el tiempo de mensajeo local-nube) son:

Duration: 0 sec
Bytes processed: 261.15 KB
Bytes billed: 10 MB
Slot milliseconds: 118

## q3_time

Principalmente igual al q1_time. En lugar de analizar los mentioned users buscando el @ en content por regex, note que el JSON posee un campo mentionedUsers, lo cual me permite analizar sus valores mas eficientemente que estar leyendo strings.

In [19]:
from q3_time import q3_time

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q3_time(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
         353646 function calls (353643 primitive calls) in 1.921 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    1.070    0.000    1.070    0.000 {orjson.loads}
   117408    0.588    0.000    1.659    0.000 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\stream_json.py:5(stream_json)
        1    0.205    0.205    1.916    1.916 C:\Users\federico.gantov\Documents\Data Arch\New folder\TEST\test\src\q3_time.py:6(q3_time)
   103403    0.050    0.000    0.050    0.000 {built-in method sys.intern}
        1    0.003    0.003    0.005    0.005 {built-in method builtins.sorted}
        3    0.002    0.001    1.919    0.640 {built-in method builtins.exec}
    15239   

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     7     52.8 MiB     52.8 MiB           1   @profile
     8                                         def q3_time(file_path: str) -> List[Tuple[str, int]]:
     9                                             """El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que
    10                                             registra cada uno de ellos."""
    11     52.8 MiB      0.0 MiB           1       if file_path is None or file_path == "":
    12                                                 print("Error: File path vacio.")
    13                                                 return []
    14                                             # Un limite para que se detenga la lectura del archivo si hay demasiados errores
    15                                             # maxErrorCount se podria poner como variable opcional, o obtenerla de una variable de entorno 
    16     52.8 MiB      0.0 MiB           1       errorCount = 0
    17     52.8 MiB      0.0 MiB           1       maxErrorCount = 100
    18
    19                                             # Utilizaremos un dict para ir creando una key por cada user, y para el almacenar el contador  
    20                                             # Esta estructura permite un rapido acceso a los datos cuando se requiera cargar mas en el mismo user
    21     52.8 MiB      0.0 MiB           1       user_counts = defaultdict(int)
    22
    23     56.7 MiB      3.2 MiB      117408       for tweet in stream_json(file_path):
    24     56.7 MiB      0.0 MiB      117407           try:
    25     56.7 MiB      0.0 MiB      117407               mentionedUsers = tweet["mentionedUsers"]
    26     56.7 MiB      0.0 MiB      117407               if mentionedUsers is not None:
    27     56.7 MiB      0.0 MiB      141437                   for user in mentionedUsers:
    28                                                             # Utilizamos intern para reducir el uso de memoria (aunque es minima la mejora)
    29                                                             # Incrementamos el contador de users
    30     56.7 MiB      0.8 MiB      103403                       user_counts[intern(user["username"])] += 1
    31
    32                                                 except Exception as e:
    33                                                     if isinstance(e, KeyError):
    34                                                         print(f"Error: Key no encontrado en tweet. Saltando linea. {e}")
    35                                                     elif isinstance(e, ValueError):
    36                                                         print(f"Error: Formato invalido en tweet. Saltando linea. {e}")
    37                                                     elif isinstance(e, TypeError):
    38                                                         print(f"Error: Tipo invalido en tweet. Saltando linea. {e}")
    39                                                     else:
    40                                                         print(f"Error: {e}")
    41                                                     errorCount += 1
    42                                                     if (errorCount >= maxErrorCount):
    43                                                         print(f"Demasiados errores, abortando...")
    44                                                         break
    45
    46                                             # Ordenamos los users por cantidad y tomamos los 10 primeros, y retornamos en el formato pedido
    47     57.6 MiB      0.9 MiB       30479       top_users = sorted(user_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    48     57.6 MiB      0.0 MiB           1       return top_users

In [ ]:
## q3_memory

La misma idea que q1_memory.

In [20]:
from q3_memory import q3_memory

# Profiler
pr = cProfile.Profile()
pr.enable()
result = q3_memory(file_path)
print(result)
pr.disable()
s = io.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
         17779 function calls (17612 primitive calls) in 4.017 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    1.645    0.183    1.645    0.183 {method 'read' of '_ssl._SSLSocket' objects}
        2    1.587    0.794    2.241    1.121 C:\Users\federico.gantov\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\ssl_.py:493(_ssl_wrap_socket_impl)
        2    0.654    0.327    0.654    0.327 {method 'do_handshake' of '_ssl._SSLSocket' objects}
        1    0.028    0.028    0.028    0.028 C:\Users\federico.gantov\AppData\Local\Programs\Python\Python312\Lib\site-packages\rsa\pkcs1.py:320(sign)
        2    0.026    0.013    0.026    0.013 {built-in method builtins.p

Los resultados en la consola de GCP (que ignoran el tiempo de mensajeo local-nube) son:

Duration: 0 sec
Bytes processed: 1.27 MB
Bytes billed: 10 MB
Slot milliseconds: 272